In [23]:
import json
from collections import defaultdict

def get_response(fname):
    with open(fname, 'r') as f:
        data = f.read()

    response = json.loads(data)
    response.pop('page_id')

    pid_resp = defaultdict(dict)
    pass_resp = defaultdict(str)
    for k, v in response.items():
        if 'response' in k:
            temp = k.split("_")
            pid = temp[0].strip('p')
            v = json.loads(v)
            pid_resp[pid].update(v)
        else:
            pass_resp[k] = v
    return [pass_resp, pid_resp]

In [26]:
import os
fpath = './response'
for i, fname in enumerate(os.listdir(fpath)):
    pss, pid = get_response(os.path.join(fpath, fname))
    print(i+1, pss)


1 defaultdict(<class 'str'>, {'password2': 'blahblahblah!', 'password1': 'blahblahblah'})
2 defaultdict(<class 'str'>, {'password1': 'aa', 'password2': 'dddd', 'password2_recall2': 'dd'})
3 defaultdict(<class 'str'>, {'password1': 'aa', 'password2': 'pop'})
4 defaultdict(<class 'str'>, {'password2': '12', 'password1': '12'})
5 defaultdict(<class 'str'>, {'password1': 'aa', 'password2': 'wsqsas', 'password2_recall2': 'wsawqs', 'password1_recall2': 'qs'})
6 defaultdict(<class 'str'>, {'password2': 's', 'password1': 'qq', 'password2_recall2': 'e', 'password1_recall2': 'ss'})
7 defaultdict(<class 'str'>, {'password1': 'Test123', 'password2': 'Test123'})
8 defaultdict(<class 'str'>, {'password1': 'password', 'password2': 'password'})
9 defaultdict(<class 'str'>, {'password1': 'test123', 'password2': 'test123'})
10 defaultdict(<class 'str'>, {})
